# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
# load in the csv from part 1 as a dataframe
df = pd.read_csv("weather_data.csv")
df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Windspeed,Cloudiness,Date,Country
0,Rikitea,-23.12,-134.97,74.588,80,8.78,100,1602982257,PF
1,Vaini,-21.20,-175.20,78.800,83,8.20,75,1602982258,TO
2,Blyznyuky,48.86,36.56,60.728,89,7.10,100,1602982258,UA
3,Dunedin,-45.87,170.50,53.006,70,0.89,53,1602982259,NZ
4,Abu Dhabi,24.47,54.37,77.000,57,1.50,0,1602982162,AE


In [4]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# set up plot
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(df[["Latitude", "Longitude"]], 
                                 weights=df["Humidity"], 
                                 max_intensity=100) 

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# filter down to ideal conditions
filtered_df = df[df["Max Temperature"] < 70]
filtered_df = filtered_df[filtered_df["Max Temperature"] > 65]

filtered_df = filtered_df[filtered_df["Humidity"] < 75]
filtered_df = filtered_df[filtered_df["Humidity"] > 20]

filtered_df = filtered_df[filtered_df["Windspeed"] < 8]
filtered_df = filtered_df[filtered_df["Windspeed"] > 2]
hotel_df = filtered_df[filtered_df["Cloudiness"] < 50]

# check number of resulting ideal cities
len(hotel_df)

9

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# make list of (lat, long) pairs
lats = hotel_df['Latitude'].tolist()
lngs = hotel_df['Longitude'].tolist()
locs = list(zip(*[lats, lngs]))

In [8]:

hotels = []
for i in range(len(hotel_df)):
    coordinates = f"{locs[i][0]}, {locs[i][1]}"
    target_search = "hotel"
    radius = 5000

    # set up a parameters dictionary
    params = {
        "location": coordinates,
        "keyword": target_search,
        "radius": radius,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    if response.status_code > 399:
        print(response.status_code)
        break
    
    hotels.append(response.json())

# now we have a list of hotel results for each location

In [9]:
# get the first hotel name for each place
names = [h['results'][0]['name'] for h in hotels]
    
# put names into df
hotel_df["Hotel Name"] = names
hotel_df

<ipython-input-9-9e7879923177>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = names


,City,Latitude,Longitude,Max Temperature,Humidity,Windspeed,Cloudiness,Date,Country,Hotel Name
88,Margate,-30.86,30.37,67.586,73,3.23,0,1602982282,ZA,St Michaels Sands Hotel and Restaurant
108,Saint-Leu,-21.15,55.28,69.800,64,3.10,12,1602982287,RE,Iloha Seaview Hotel
120,Saint-Philippe,-21.36,55.77,69.800,64,3.10,38,1602982291,RE,Les Embruns Du Baril
269,Camacha,33.08,-16.33,69.800,68,6.20,20,1602982330,PT,Vila Baleira Resort
300,Kaeo,-35.10,173.78,69.008,62,2.24,0,1602982338,NZ,Pacific Harbour Lodge
301,Callaway,30.15,-85.57,69.008,56,2.10,1,1602982339,US,Holiday Inn Express & Suites Panama City-Tyndall
327,Rawson,-43.30,-65.10,68.432,31,7.60,0,1602982211,AR,Hotel Punta León
368,Ngunguru,-35.62,174.50,69.008,62,2.24,23,1602982359,NZ,Quality Hotel Oceans Tutukaka
457,Russell,-35.26,174.12,69.008,62,2.24,0,1602982383,NZ,Hananui Lodge & Apartments


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# create symbol layer
symbols = gmaps.symbol_layer(locations,
                             info_box_content=hotel_info,
                             fill_color='rgba(0, 150, 0, 0.4)',
                             stroke_color='rgba(0, 0, 150, 0.4)', scale=2,)
# Add the layer to the map
fig.add_layer(symbols)

# show figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…